In [1]:
require 'countries'
require 'numo/narray'
require 'numo/linalg'

true

# 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
空文字列となったトークンは削除
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [11]:
doc = []
pat = "[#{["\\", "\\.", "\\,", "\\!", "\\?", "\\;", "\\:", "\\(", "\\)", "\\[", "\\]", "\\'", "\""].join()}]"

# File.foreach('../data/enwiki_small').with_index do |line, index|
File.foreach('../data/enwiki-20150112-400-r100-10576.txt').with_index do |line, index|
  p index if index % 50000 == 0
  sentence = line.chomp.split.map {|token|
    token.
      sub(/^#{pat}/, '').
      sub(/#{pat}$/, '').
      gsub(/#{pat}/, '') # ズル
    }.
    reject{|token| token.empty?}
  
  doc << sentence.join("\s") if sentence.size > 5
end

1

0
50000
100000
150000
200000
250000


1

# 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は“United States”，イギリスは“United Kingdom”と表現されるが，“United”や“States”，“Kingdom”という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．
インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，“United States”は“United_States”，“Isle of Man”は“Isle_of_Man”になるはずである．

In [13]:
original = ISO3166::Country.all.map{|country| country.name.sub(/\s\(.*?\)\s/, '')}.select{|name| name.match(/\s/)}
under_scored = original.map{|name| name.gsub(/\s/, '_')}

doc.each_with_index do |sentence, index|
  p index if index % 50000 == 0
  original.size.times do |i|
    sentence.gsub!(original[i-1], under_scored[i-1])
  end
end

1

0
50000
100000
150000


1

# 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．
- ある単語tの前後d単語を文脈語cとして抽出する（ただし，文脈語に単語tそのものは含まない）
- 単語tを選ぶ度に，文脈幅dは{1,2,3,4,5}の範囲でランダムに決める．

In [22]:
def window
  (Random.rand * 4.9999).to_i + 1
end

output = open("../data/context.tsv", "w")

doc.each_with_index do |words, index|
  p index if index % 50000 == 0
  words = words.split(/\s|\n/)
  
  words.size.times.each do |i|
    w = window

    start_from = [0, i - w].max
    end_to = [words.size-1, i + w].min

    contexts = words[start_from...i] + words[(i+1)..end_to]
    contexts.each do |context|
      output.puts "#{words[i]}\t#{context}"
    end
  end
end

output.close

1

0
50000
100000
150000


1

# 83. 単語／文脈の頻度の計測

82の出力を利用し，以下の出現分布，および定数を求めよ．

f(t,c)f(t,c): 単語ttと文脈語ccの共起回数
f(t,∗)f(t,∗): 単語ttの出現回数
f(∗,c)f(∗,c): 文脈語ccの出現回数
NN: 単語と文脈語のペアの総出現回数

In [26]:
count_t = Hash.new{|hash, key| hash[key] = 0}
count_c = Hash.new{|hash, key| hash[key] = 0}
count_t_c = Hash.new{|hash, key| hash[key] = Hash.new{|hash2, key2| hash2[key2] = 0}}
n = 0.0

File.foreach("../data/context.tsv").with_index do |line, index|
  p index if index % 10000000 == 0
  t, c = line.split
  count_t[t] += 1
  count_c[c] += 1
  count_t_c[t][c] += 1
  n += 1
end

1

0
10000000
20000000
30000000
40000000
50000000
60000000


1

# 84. 単語文脈行列の作成

83の出力を利用し，単語文脈行列XXを作成せよ．ただし，行列XXの各要素XtcXtcは次のように定義する．

f(t,c)≥10f(t,c)≥10ならば，Xtc=PPMI(t,c)=max{logN×f(t,c)f(t,∗)×f(∗,c),0}Xtc=PPMI(t,c)=max{log⁡N×f(t,c)f(t,∗)×f(∗,c),0}
f(t,c)<10f(t,c)<10ならば，Xtc=0Xtc=0
ここで，PPMI(t,c)PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列XXの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は00になるので，非00の要素だけを書き出せばよい．

多分NArrayに疎行列無い

In [28]:
c_token2index = {}
count_t.keys.each_with_index{|key, index| c_token2index[key] = index}
t_token2index = {}
count_t.keys.each_with_index{|key, index| t_token2index[key] = index}

mat = Numo::Float64.zeros(count_t.size, count_c.size)
count_t.keys.each do |t|
  count_c.keys.each do |c|
    if count_t_c[t][c] >= 1
      pmmi = Numo::NMath.log((n * count_t_c[t][c]) /  (count_t[t] * count_c[c]))
      mat[t_token2index[t], c_token2index[c]] = [0.0, pmmi].max
    end
  end
end

1

NoMemoryError: failed to allocate memory

# 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [10]:
Numo::Linalg.eigen(mat)

[Numo::DComplex#shape=[53]
[12.205+0i, 11.233+0i, 9.78139+0i, 9.07889+0i, 8.42578+0i, 7.72728+0i, ...], Numo::DComplex#shape=[53,53]
[[0.134642+0i, 0.202037+0i, 0.193645+0i, 0.203658+0i, 0.190824+0i, ...], 
 [-0.0724657+0i, -0.166701+0i, 0.00650275+0i, 0.0382719+0i, 0.0800707+0i, ...], 
 [-0.0875227+0i, 0.330107+0i, 0.177933+0i, 0.223125+0i, 0.163525+0i, ...], 
 [-0.267009+0i, -0.294244+0i, -0.232981+0i, -0.186589+0i, -0.0817452+0i, ...], 
 [-0.147058+0i, -0.393911+0i, -0.394805+0i, -0.407532+0i, -0.249339+0i, ...], 
 [-0.0846374+0i, -0.182303+0i, -0.194034+0i, -0.19444+0i, -0.102091+0i, ...], 
 [-0.0411155+0i, 0.195889+0i, -0.112879+0i, -0.113887+0i, -0.133322+0i, ...], 
 [0.0500948+0i, 0.302949+0i, 0.294643+0i, 0.328698+0i, 0.135477+0i, ...], 
 [-0.0335798+0i, -0.10322+0i, -0.344055+0i, -0.40663+0i, -0.188552+0i, ...], 
 [0.12376+0.0322708i, 0.226076+0.0785778i, -0.12661-0.113593i, ...], 
 [0.12376-0.0322708i, 0.226076-0.0785778i, -0.12661+0.113593i, ...], 
 [0.176495+0i, 0.191686+0i